In [ ]:
pip install pandas pyarrow

In [1]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime

In [ ]:
#archivo: "C:\Users\Usuario\Henry\PI01MLOPs\DataSet\Users_items\users_items.json.gz"

In [2]:
# Desarrollo función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(users_items):

    # Abrir el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(users_items, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [3]:
datos_users_items = []
for i  in gzip.open('users_items.json.gz'):
    datos_users_items.append(ast.literal_eval(i.decode('utf-8')))

df_users_items = pd.DataFrame(datos_users_items)


In [ ]:
df_users_items

In [5]:
df_users_items = df_users_items.dropna()

In [ ]:
df_users_items

In [ ]:
# borrar de una vez todas las columnas que no sirven
# user_id	items_count	steam_id	user_url items

In [14]:
df_users_items =  df_users_items.drop(columns=['items_count'])

In [16]:
df_users_items =  df_users_items.drop(columns=['steam_id'])

In [18]:
df_users_items =  df_users_items.drop(columns=['user_url'])

In [ ]:
df_users_items

In [20]:
df_users_items.to_parquet('users_items.parquet', engine='pyarrow')

In [ ]:
df_users_items.head()

In [ ]:
df_users_items.info()

In [23]:
# Dado que el archivo esta muy largo no desanidaré la columna hasta que el enunciado no lo indique y queda asi el archivo que ya está manejable
# Al hacer la función veo que debo desanidar columna "items" para encontrar el tiempo jugado

df_users_items_exploded = df_users_items.explode('items')

In [24]:
# Normalizo los datos y creo un nuevo data frame desanidado
df_users_items_desanidado = pd.json_normalize(df_users_items_exploded['items'].dropna())

In [25]:
# Volver a indexar
df_users_items_desanidado.reset_index(inplace=True)
df_users_items_exploded.reset_index(inplace=True)

In [26]:
#  Concateno Data Frames y borro columna original items
df_users_items_concatenado = pd.concat([df_users_items_exploded, df_users_items_desanidado], axis=1)
df_users_items_concatenado =  df_users_items_concatenado.drop(columns=['items'])

In [ ]:
df_users_items_concatenado

In [30]:
#Para disminuir espacio borro columnas que no usaré.
# Borro index	user_id	item_id	 playtime_2weeks
# Lista de columnas a borrar
columnas_a_borrar = ['index','user_id','item_id','playtime_2weeks']
#Borrar una sola columna
# del df['B']


In [31]:
df_users_items_concatenado = df_users_items_concatenado.drop(columns=columnas_a_borrar)

In [ ]:
df_users_items_concatenado

In [33]:
df_users_items = df_users_items_concatenado

In [ ]:
df_users_items

In [35]:
df_users_items.to_json('df_users_items.json',orient='records')